In [1]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

"\nimport ibmos2spark\n# @hidden_cell\ncredentials = {\n    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',\n    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',\n    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',\n    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'\n}\n\nconfiguration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'\ncos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')\n\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.getOrCreate()\n\ndf = spark.read  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')  .option('header', 'true')  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))\n"

In [2]:
from functions import *
from datetime import datetime

from pyspark.ml.feature import VectorAssembler,OneHotEncoderEstimator,QuantileDiscretizer, StringIndexer, Imputer,StandardScaler,MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import ChiSqSelector,PCA

from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
model_indexing_dir = "log_index.md"
models_dir = "models"
logs_dir = "models" 
filter_state = 'SC' # ALL if all states
model_Note = ''
log_mode=True
debug_mode=True
logger = logging(models_dir, logs_dir, "", filter_state, enabled=log_mode)

file = "data/US_Accidents_Dec19.csv"


df, sc,spark = setup_spark(file)

In [4]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [5]:
tmp = sc._conf.getAll()
logger.write2file("New Spark session", str(tmp))
print("Config:",tmp)

Config: [('spark.driver.memory', '4g'), ('spark.executor.memory', '4g'), ('spark.ui.enabled', 'true'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Spark Project'), ('spark.ui.killEnabled', 'false'), ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), ('spark.driver.host', '97901d576ca5'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.app.id', 'local-1589251920243'), ('spark.executor.instances', '1'), ('spark.master', 'local[*]'), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'client'), ('spark.driver.port', '37213'), ('spark.kryoserializer.buffer.max', '15'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.cores', '1')]


# Debug tool

In [6]:
if debug_mode == True: 
    df.createOrReplaceTempView("records")
    reviewData = spark.sql("SELECT * FROM records LIMIT 100000") 

#df,_ = df.randomSplit([1.0,0.0], 1) # Speed up the program a bit
df.count()

2925212

# 1. Define variables

In [7]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

df, colCat, colNum = setup_variables(df, sc, colLabel, colRem)  

logger.write2file("Number of rows", str(df.count()))
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

Label: ['Severity'] 
Categories: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 2. Preprocessing

## Analyse single state
#Create a model for separates states instead of all states at once.

In [8]:
if filter_state != 'ALL':
    df = df.filter(df.State == filter_state) # Lowers the dataset quite a lot
    logger.write2file("Specified state",str(filter_state))
else:
    logger.write2file("No state specified","")

## Modify time
#Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [9]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)
logger.write2file("Dataset after modifying UTC timestamp", str(df.take(1)))

+-----+--------+-------------------+---------+----------+------------+------+---------+----+----+--------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+
|  TMC|Severity|         Start_Time|Start_Lat| Start_Lng|Distance(mi)|Number|   Street|Side|City|  County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunri

## Quantiles function
#Alternative solution to get position 
#Remove City, Country, State, Zipcode, Airport_Code
#In order to reduce dimensionality feature crossing will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data

In [10]:
# TODO - Calculate how many buckets needed (Bins)
discretizer_Lat = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lat", outputCol="Start_Lat_disc")
discretizer_Lng = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lng", outputCol="Start_Lng_disc")

# Into categorical values
indexer_cord = [StringIndexer(inputCol=c + "_disc", outputCol=c+"_IDX") for c in ["Start_Lat","Start_Lng"]]

# One-hot (feature crossing)
encoder_cord = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexer_cord], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexer_cord])

pipeline_cord = Pipeline(stages=[discretizer_Lat,discretizer_Lng, *indexer_cord, encoder_cord])
preprocessed_cord = pipeline_cord.fit(df).transform(df)

position = VectorAssembler(inputCols=["Start_Lat_IDX_vec","Start_Lng_IDX_vec"], outputCol="position")

df = position.transform(preprocessed_cord)

In [11]:
df.show(1)
logger.write2file("Dataset after adding quantiles",str(df.take(1)))

+-----+--------+-------------------+---------+----------+------------+------+---------+----+----+--------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+--------------+--------------+-------------+-------------+-----------------+-----------------+--------------------+
|  TMC|Severity|         Start_Time|Start_Lat| Start_Lng|Distance(mi)|Number|   Street|Side|City|  County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Cr

## Cast data 
#Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values. Data removed caused to big feature vector, contained too #many NaN values or were converted into another form.

In [12]:
# Could be handles as separated lists and then sent to a UDF for processing but since low number of features and easier to visualize this select is kept.
df = df.select(
        col('Severity').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        col('position'),
        col('Distance(mi)').cast('double'),
        col('Side').cast('string'),   
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'), 
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'), 
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

colLabel = ["Severity"]

colCat, colNum = createNewClasses(df, sc, colLabel)
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

Label: ['Severity'] 
Categories: ['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Clean data

### Recheck the missing values
#Check so that the output contains 0 missing values

In [13]:
df.persist()
printMissingValues(df,logger)
df.unpersist()

Missing values[Row(Severity=0, Start_Hour=1391, Start_Month=1391, Weather_Hour=2257, Weather_Month=2257, position=0, Distance(mi)=0, Side=0, Temperature(F)=2836, Wind_Chill(F)=89759, Humidity(%)=2865, Pressure(in)=2494, Visibility(mi)=3137, Wind_Direction=2535, Wind_Speed(mph)=25585, Weather_Condition=3079, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=0, Civil_Twilight=0, Nautical_Twilight=0, Astronomical_Twilight=0)]


DataFrame[Severity: int, Start_Hour: string, Start_Month: string, Weather_Hour: string, Weather_Month: string, position: vector, Distance(mi): double, Side: string, Temperature(F): double, Wind_Chill(F): double, Humidity(%): double, Pressure(in): double, Visibility(mi): double, Wind_Direction: string, Wind_Speed(mph): double, Weather_Condition: string, Amenity: string, Bump: string, Crossing: string, Give_Way: string, Junction: string, No_Exit: string, Railway: string, Roundabout: string, Station: string, Stop: string, Traffic_Calming: string, Traffic_Signal: string, Turning_Loop: string, Sunrise_Sunset: string, Civil_Twilight: string, Nautical_Twilight: string, Astronomical_Twilight: string]

### Remove data with occurance less than 1%
#Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [14]:
df.persist()
original_rows = df.count()
n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)
df_filtered = weather_freq.filter(weather_freq['count'] > n)
filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()
df = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df_filtered.show()
removed_rows = original_rows - df.count()

df.unpersist()
print("Rows removed:",removed_rows)

logger.write2file("Weather condition", str(df_filtered.take(df_filtered.count())) + "\nRows removed: " + str(removed_rows))

+--------------------+-----+
|   Weather_Condition|count|
+--------------------+-----+
|               Clear|44274|
|                Fair|25503|
|            Overcast|15699|
|       Mostly Cloudy|15139|
|       Partly Cloudy|10433|
|    Scattered Clouds| 9870|
|          Light Rain| 7139|
|              Cloudy| 5475|
|                null| 3079|
|                Rain| 2650|
|                 Fog| 1661|
|          Heavy Rain| 1008|
|                Haze|  780|
|       Light Drizzle|  610|
|             Drizzle|  317|
|Thunder in the Vi...|  312|
|      Patches of Fog|  303|
|        Thunderstorm|  271|
|Light Thunderstor...|  207|
|             T-Storm|  199|
+--------------------+-----+
only showing top 20 rows

Rows removed: 4152


# Remove if distict values in category equals 1

In [15]:
df.persist()
tmp = [c for c in [*colCat, "position"] if df.select(countDistinct(c)).collect()[0][0] <= 1] 
df.unpersist()

[colCat.remove(c) for c in tmp]
print("Dropping columns with 1 class:", tmp)
logger.write2file("Dropping columns with 1 class", str(tmp))

Dropping columns with 1 class: ['Turning_Loop']


## Prepare Pipeline

In [16]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='skip') for c in colCat]

encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers])

## Preprocessing - Pipeline

In [17]:
# Categorical values

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

pipeline = Pipeline(stages=[numPipeline, catPipeline])

preprocessed_df = pipeline.fit(df).transform(df)

In [18]:
preprocessed_df.select("position","scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(1,False)

+------------------------+----------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------+---------------------+------------+----------------------+-------------------------+---------------+-------------+----------------+----------------+----------------+---------------+---------------+------------------+---------------+-------------+-----------------------+----------------------+----------------------+----------------------+-------------------------+-----------------------------+
|position                |scaledFeatures                                                                                                              |Start_Hour_IDX_vec|Start_Month_IDX_vec|Weather_Hour_IDX_vec|Weather_Month_IDX_vec|Side_IDX_vec|Wind_Direction_IDX_vec|Weather_Condition_IDX_vec|Amenity_IDX_vec|Bump_IDX_vec |Crossing_IDX_vec|Give_Way_IDX_vec|Junction_IDX_vec|No_Exit_IDX_vec|R

## Combine features

In [19]:
va2 = VectorAssembler(inputCols=["position","scaledFeatures",*[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

df = va2.transform(preprocessed_df)
df.persist()
df = df.withColumn('label', col("Severity"))
df_features = df.withColumn('features', df.final_features).select("features","label")
df.unpersist()
df_features.show(1, False)

print(type(df_features))

logger.write2fileModel(df_features, "df_features")

logger.write2file("Feature set size", str(df_features.count()) + "\n\n__Feature vector and label:__\n" + str(df_features.take(1)))
logger.write2file("Number of rows", str(df_features.count()))
print("Feature set size: ",df_features.count())

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                       |label|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Count categorical values

In [20]:
# Checking how many classes that can be used
df.persist()
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat, "position"]] 
df.unpersist()
print("Unique column values:", tmp)

logger.write2file("Unique column values", str(tmp))

Unique column values: [Row(Start_Hour=24), Row(Start_Month=12), Row(Weather_Hour=24), Row(Weather_Month=12), Row(Side=2), Row(Wind_Direction=24), Row(Weather_Condition=21), Row(Amenity=2), Row(Bump=2), Row(Crossing=2), Row(Give_Way=2), Row(Junction=2), Row(No_Exit=2), Row(Railway=2), Row(Roundabout=2), Row(Station=2), Row(Stop=2), Row(Traffic_Calming=2), Row(Traffic_Signal=2), Row(Sunrise_Sunset=2), Row(Civil_Twilight=2), Row(Nautical_Twilight=2), Row(Astronomical_Twilight=2), Row(position=5538)]


# 3. Feature importance

## PCA
#In order to understand how much the variance affect the dataset we check with PCA. Try to get above 90%

In [21]:
k=250
pca = PCA(k=k, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_features)
pca_df = pca_model.transform(df_features)
#chi_model[0].pca_features

print("PCA - Feature Variance:","Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+str(k)+"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
logger.write2file("PCA - Feature variance", "Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+ str(k) +"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
logger.write2fileModel(pca_df, "pca_df")

PCA - Feature Variance: Top 50:
[0.05766229 0.03530102 0.02757442 0.0256291  0.02532432 0.02461963
 0.02438295 0.02273854 0.02122041 0.01998778 0.01843186 0.01785771
 0.01713528 0.0167578  0.01634119 0.01592616 0.01538732 0.01529416
 0.01491239 0.0142619  0.0137218  0.01306225 0.01261786 0.01210671
 0.01160655 0.01121158 0.01078696 0.01064676 0.01004623 0.0098263
 0.00884415 0.0086728  0.0082641  0.00813522 0.00739017 0.00713196
 0.00700715 0.00687807 0.006521   0.00578096 0.00523802 0.00515462
 0.00501049 0.00495673 0.00472208 0.00464995 0.00458716 0.0044969
 0.00439275 0.00420986]
Number of items: 250
Sum of variance: 0.9734968859053912
Saving into: models/SC_20200512025158/pca_df
Saving through format 1


## ChiSqSelector
#Check top 100 which of the values in the feature vector

In [22]:
selector = ChiSqSelector(numTopFeatures=k, 
                         labelCol='label', 
                         featuresCol='features', 
                         outputCol="selectedFeatures",
                         selectorType='numTopFeatures', 
                         percentile=0.1, 
                         fpr=0.05, fdr=0.05, fwe=0.05)

chi_model = selector.fit(df_features)
chi_df = chi_model.transform(df_features)

logger.write2file("Top selected features according to ChiSqSelector", str(chi_model.selectedFeatures)+ "\nNumber of features: " + str(k) + "\nExample data:\n"+str(chi_df.take(5)))
logger.write2fileModel(chi_df, "chi_df")

print("Top selected features according to ChiSqSelector:", chi_model.selectedFeatures)
print("Transformed selected features:",chi_df.head().selectedFeatures)
print("Transformed feature vector:", chi_df.show(10))

Saving into: models/SC_20200512025158/chi_df
Saving through format 1
Top selected features according to ChiSqSelector: [4, 7, 8, 13, 15, 16, 19, 23, 25, 27, 28, 29, 32, 38, 45, 47, 53, 58, 59, 61, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 77, 79, 81, 84, 86, 88, 91, 93, 99, 101, 102, 104, 105, 107, 109, 114, 115, 120, 121, 124, 128, 129, 130, 132, 133, 134, 137, 142, 143, 146, 148, 149, 151, 152, 153, 155, 157, 158, 159, 160, 166, 169, 170, 172, 173, 174, 179, 180, 182, 183, 188, 193, 194, 195, 196, 198, 199, 200, 202, 203, 204, 224, 225, 226, 227, 258, 259, 260, 261, 273, 275, 298, 300, 301, 302, 317, 319, 321, 323, 325, 326, 328, 329, 330, 331, 332, 305, 110, 57, 122, 222, 87, 156, 201, 223, 136, 54, 106, 184, 255, 82, 126, 64, 307, 0, 144, 197, 312, 154, 140, 244, 211, 257, 304, 50, 62, 89, 34, 1, 178, 119, 44, 308, 108, 167, 56, 112, 22, 43, 74, 164, 9, 147, 254, 296, 90, 138, 139, 294, 297, 267, 233, 250, 246, 221, 218, 186, 98, 215, 209, 55, 36, 287, 12, 249, 220, 192, 285, 219

# 4. Machine learning

In [23]:
# Use the chi_df for training since it is reduced
#trainSet, testSet = df_features.randomSplit([0.8 ,0.2], 1)
trainSet, testSet = chi_df.randomSplit([0.8 ,0.2], 1)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [24]:
trainSet.show(2,False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                    |label|selectedFeatures           

## Logistic regression

In [25]:
lr = LogisticRegression(labelCol="label", featuresCol="selectedFeatures")

paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1,0.01]) \
                .addGrid(lr.maxIter, [10]) \
                .addGrid(lr.elasticNetParam, [0.6]) \
                .build()
try: 
    model, _ = evaluateModel(lr, paramGrid, "LR_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)
except Exception as e:
    logger.write2file("Error:", str(e))
    print(e)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/SC_20200512025158/LR_Model
'function' object has no attribute 'format'
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    2|(333,[0,113,198,1...|
|       2.0|    2|(333,[0,113,199,2...|
|       2.0|    2|(333,[0,163,199,2...|
|       2.0|    2|(333,[0,185,199,2...|
|       2.0|    2|(333,[3,112,199,2...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.7612227624648832 
Parameters
 {Param(parent='LogisticRegression_3ee5d4c96c80', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_3ee5d4c96c80', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_3ee5d4c96c80', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.81      0.98      0.89     22104
           3       0.73      0.21      0.33      6210
           4       0.00      0.00      0.00       175

    accuracy                           0.81     28495
   macro avg       0.39      0.30      0.30     28495
weighted avg       0.79      0.81      0.76     28495

Confusion matrix:
[[    0     6     0     0]
 [    0 21665   439     0]
 [    0  4880  1330     0]
 [    0   130    45     0]]


## Decision tree

In [26]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="selectedFeatures") 

paramGrid = ParamGridBuilder().build()

_, _ = evaluateModel(dt, paramGrid, "DT_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/SC_20200512025158/DT_Model
'function' object has no attribute 'format'
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    2|(333,[0,113,198,1...|
|       2.0|    2|(333,[0,113,199,2...|
|       2.0|    2|(333,[0,163,199,2...|
|       2.0|    2|(333,[0,185,199,2...|
|       2.0|    2|(333,[3,112,199,2...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.7510268211255935 
Parameters
 {}
Classification report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.80      0.99      0.89     22104
           3       0.81      0.18      0.29      6210
           4       0.00      0.00      0.00       175

    accuracy                           0.80     28495
   macro avg       0.40      0.29      0

## Random forest

In [27]:
from sklearn.metrics import plot_confusion_matrix

rf = RandomForestClassifier(labelCol="label", featuresCol="selectedFeatures") #numTrees=10
paramGrid = ParamGridBuilder() \
                .addGrid(rf.numTrees,[15]) \
                .build()

_, _ = evaluateModel(rf, paramGrid, "RF_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/SC_20200512025158/RF_Model
'function' object has no attribute 'format'
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    2|(333,[0,113,198,1...|
|       2.0|    2|(333,[0,113,199,2...|
|       2.0|    2|(333,[0,163,199,2...|
|       2.0|    2|(333,[0,185,199,2...|
|       2.0|    2|(333,[3,112,199,2...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.6777369194606085 
Parameters
 {Param(parent='RandomForestClassifier_00d454841bb0', name='numTrees', doc='Number of trees to train (>= 1).'): 15}
Classification report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.78      1.00      0.87     22104
           3       0.00      0.00      0.00      6210
           4       0.00      0.00    

In [28]:
logger.write2file("Program finished!", "")

In [29]:
logger.write2file("New model folder created", \
               "Model finished: Yes" + \
               "\nFolder name: " + logger.timeSignature + \
               "\nState: "+ filter_state + \
               "\nLogs directory: " + logger.logs_dir +  \
               "\nFile: " + file + \
               "\nNote: " + model_Note, \
               logs_dir=model_indexing_dir \
              )
